# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering using **scikit-learn and Pandas** libraries.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!
    
## Import ML Libraries

Import the required libraries

In [1]:
import numpy as np
import pandas as pd
import math
import string

# Data preparation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [2]:
products = pd.read_csv('amazon_baby.csv')

Now, let us see a preview of what the dataset looks like.

In [3]:
products.head(5)

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


## Perform text cleaning

Filling the NaN values with the empty string in 'review' column.

In [4]:
products['review'] = products['review'].fillna('')

#products = products.dropna(subset=['review']) # Dropping the rows with NaN value of 'review'
#products[38:39] #
#products['review'].isnull().sum()  #Count the number of rows with 'review' value NaN.
#len(products)  #Get the length of the dataframe

Now, we will perform simple data transformations:

Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [5]:
def remove_punctuation(text):
    return text.translate(str.maketrans({key: None for key in string.punctuation})) 

products['review_clean'] = products['review'].apply(remove_punctuation)

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [6]:
products = products[products['rating'] != 3]
len(products)

166752

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [7]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products.head(5)

,name,review,rating,review_clean,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...,1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,When the Binky Fairy came to our house we didn...,1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

Read the training data and test data from the json files.

In [8]:
train_indices = pd.read_json('module-2-assignment-train-idx.json')
test_indices = pd.read_json('module-2-assignment-test-idx.json')

train_data = products.iloc[train_indices[0].tolist(), :]
test_data = products.iloc[test_indices[0].tolist()]

## Build the word count vector for each review
We will now compute the word count for each word that appears in the reviews. A vector consisting of word counts is often referred to as **bag-of-word features**. Since most words occur in only a few reviews, word count vectors are sparse. For this reason, scikit-learn and many other tools use sparse matrices to store a collection of word count vectors. Refer to appropriate manuals to produce sparse word count vectors. General steps for extracting word count vectors are as follows:

* Learn a vocabulary (set of all words) from the training data. 
* Only the words that show up in the training data will be considered for feature extraction.
* Compute the occurrences of the words in each review and collect them into a row vector.
* Build a sparse matrix where each row is the word count vector for the corresponding review. Call this matrix **train_matrix**.
* Using the same mapping between words and columns, convert the test data into a sparse matrix **test_matrix**.
* The following cell uses CountVectorizer in scikit-learn. Notice the **token_pattern** argument in the constructor.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
# Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
#train_matrix
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review_clean'])

# Train a sentiment classifier with logistic regression

Learn a logistic regression classifier using the training data. If you are using scikit-learn, you should create an instance of the LogisticRegression class and then call the method fit() to train the classifier. This model should use the sparse word count matrix (train_matrix) as features and the column sentiment of train_data as the target. Use the default values for other parameters. Call this model sentiment_model.

In [10]:
from sklearn import linear_model
sentiment_model = linear_model.LogisticRegression().fit(train_matrix, train_data['sentiment'])

There should be over 100,000 coefficients in this sentiment_model. Recall from the lecture that positive weights w_j correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. Calculate the number of positive (>= 0, which is actually nonnegative) coefficients.

In [11]:
weights = sentiment_model.coef_
type(weights)

numpy.ndarray

In [12]:
num_positive_weights = np.sum(weights >= 0)
num_negative_weights = np.sum(weights < 0)

print("Number of positive weights: %s " % num_positive_weights)
print("Number of negative weights: %s " % num_negative_weights)

Number of positive weights: 87194 
Number of negative weights: 34518 


**Quiz Question:** How many weights are >= 0?

## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [13]:
sample_test_data = test_data[10:13]
print(sample_test_data['rating'])
sample_test_data

59    5
71    2
91    1
Name: rating, dtype: int64


,name,review,rating,review_clean,sentiment
59,Our Baby Girl Memory Book,Absolutely love it and all of the Scripture in...,5,Absolutely love it and all of the Scripture in...,1
71,Wall Decor Removable Decal Sticker - Colorful ...,Would not purchase again or recommend. The dec...,2,Would not purchase again or recommend The deca...,-1
91,New Style Trailing Cherry Blossom Tree Decal R...,Was so excited to get this product for my baby...,1,Was so excited to get this product for my baby...,-1


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [14]:
sample_test_data.iloc[0,1]

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [15]:
sample_test_data.iloc[1,1]

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores**. For each row, the score (or margin) is a number in the range (-inf, inf). Use a pre-built function in your tool to calculate the score of each data point in sample_test_data. In scikit-learn, you can call the decision_function() function.

Hint: You'd probably need to convert sample_test_data into the sparse matrix format first.

In [16]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores = sentiment_model.decision_function(sample_test_matrix)
print(scores)

[  5.60129879  -3.17005306 -10.42408041]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [17]:
sentiment_model.predict(sample_test_matrix)

array([ 1, -1, -1])

**Checkpoint**: Make sure your class predictions match with the one obtained from GraphLab Create.

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [18]:
sentiment_model.predict_proba(sample_test_matrix)

array([[  3.67947553e-03,   9.96320524e-01],
       [  9.59691637e-01,   4.03083629e-02],
       [  9.99970292e-01,   2.97075268e-05]])

In [19]:
probablities = 1/(1 +np.exp(-scores))
print(probablities)

[  9.96320524e-01   4.03083629e-02   2.97075268e-05]


**Checkpoint**: Make sure your probability predictions match the ones obtained from GraphLab Create.

** Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review?

# Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use sklearn.linear_model.LogisticRegression to form predictions on all of the test data points.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on test_data using the sentiment_model.
2.  Sort the data according to those predictions and pick the top 20. 

In [20]:
test_data_matrix = vectorizer.transform(test_data['review_clean'])
test_data_scores = sentiment_model.decision_function(test_data_matrix)
test_data_probablities = 1/(1 + np.exp(-test_data_scores))

In [21]:
test_data['test_data_probablities'] = test_data_probablities

/Users/anjali_chadha/anaconda3/envs/MLWashington/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [22]:
test_data_sorted = test_data.sort(['test_data_probablities'], ascending=False)

/Users/anjali_chadha/anaconda3/envs/MLWashington/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this exercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [23]:
test_data_sorted_ascending = test_data.sort(['test_data_probablities'])
test_data_sorted_ascending.head(20)

/Users/anjali_chadha/anaconda3/envs/MLWashington/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,name,review,rating,review_clean,sentiment,test_data_probablities
16042,Fisher-Price Ocean Wonders Aquarium Bouncer,We have not had ANY luck with Fisher-Price pro...,2,We have not had ANY luck with FisherPrice prod...,-1,8.470331e-16
120209,Levana Safe N'See Digital Video Baby Monitor w...,This is the first review I have ever written o...,1,This is the first review I have ever written o...,-1,1.600278e-15
77072,Safety 1st Exchangeable Tip 3 in 1 Thermometer,I thought it sounded great to have different t...,1,I thought it sounded great to have different t...,-1,8.133312e-14
48694,Adiri BPA Free Natural Nurser Ultimate Bottle ...,I will try to write an objective review of the...,2,I will try to write an objective review of the...,-1,9.829438e-14
155287,VTech Communications Safe &amp; Sounds Full Co...,"This is my second video monitoring system, the...",1,This is my second video monitoring system the ...,-1,1.930520e-13
94560,The First Years True Choice P400 Premium Digit...,Note: we never installed batteries in these un...,1,Note we never installed batteries in these uni...,-1,3.318610e-13
53207,Safety 1st High-Def Digital Monitor,We bought this baby monitor to replace a diffe...,1,We bought this baby monitor to replace a diffe...,-1,3.275794e-11
81332,Cloth Diaper Sprayer--styles may vary,I bought this sprayer out of desperation durin...,1,I bought this sprayer out of desperation durin...,-1,3.341727e-11
10677,Philips AVENT Newborn Starter Set,"It's 3am in the morning and needless to say, t...",1,Its 3am in the morning and needless to say thi...,-1,9.495650e-11
113995,Motorola Digital Video Baby Monitor with Room ...,DO NOT BUY THIS BABY MONITOR!I purchased this ...,1,DO NOT BUY THIS BABY MONITORI purchased this m...,-1,9.603219e-11


**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

In [24]:
test_data['predicted_test_results'] = sentiment_model.predict(test_data_matrix)
print("Count of test data", test_data.size)
test_matching_results_data = test_data[test_data['predicted_test_results'] == test_data['sentiment']]
print("Count of the test data with matching results ", test_matching_results_data.size)

Count of test data 233352
Count of the test data with matching results  217553


/Users/anjali_chadha/anaconda3/envs/MLWashington/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [25]:
accuracy = 217553/233352
print(accuracy)

0.9322954163666907


**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76).

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better?

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subset of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [26]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [27]:
len(significant_words)

20

Compute a new set of word count vectors using only these words. The CountVectorizer class has a parameter that lets you limit the choice of words when building word count vectors:

In [28]:
vectorizer_word_subset = CountVectorizer(vocabulary=significant_words) # Limit the vocabulary
train_matrix_word_subset = vectorizer_word_subset.fit_transform(train_data['review_clean'])
test_matrix_word_subset = vectorizer_word_subset.transform(test_data['review_clean'])

Compute word count vectors for the training and test data and obtain the sparse matrices train_matrix_word_subset and test_matrix_word_subset, respectively.

## Train a logistic regression model on a subset of data

Now build a logistic regression classifier with **train_matrix_word_subset** as features and **sentiment** as the target. Call this model **simple_model**.

In [29]:
simple_model = linear_model.LogisticRegression().fit(train_matrix_word_subset, train_data['sentiment'])

Let us inspect the weights (coefficients) of the **simple_model**. First, build a table to store (word, coefficient) pairs. If you are using SFrame with scikit-learn, you can combine words with coefficients by running

In [30]:
coefficients = simple_model.coef_[0]
word_2_coeff_map = dict(zip(significant_words, coefficients))

In [31]:
word_2_coeff_map

{'able': 0.19090857206454992,
 'broke': -1.6515763449652578,
 'car': 0.05885467115273034,
 'disappointed': -2.3482982195022442,
 'easy': 1.1925382734895196,
 'even': -0.51137963179882184,
 'great': 0.9439995905719335,
 'less': -0.20956286453453807,
 'little': 0.52018576271821038,
 'love': 1.3636897593113817,
 'loves': 1.673073892593705,
 'money': -0.89803073771463382,
 'old': 0.085512779463054259,
 'perfect': 1.5098124766922481,
 'product': -0.32055623673442618,
 'return': -2.1093310903186402,
 'waste': -2.0336986139400253,
 'well': 0.50376045776762213,
 'work': -0.62116877364155332,
 'would': -0.36216674227373841}

Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [32]:
import operator
sorted(word_2_coeff_map.items(), key= operator.itemgetter(1))

[('disappointed', -2.3482982195022442),
 ('return', -2.1093310903186402),
 ('waste', -2.0336986139400253),
 ('broke', -1.6515763449652578),
 ('money', -0.89803073771463382),
 ('work', -0.62116877364155332),
 ('even', -0.51137963179882184),
 ('would', -0.36216674227373841),
 ('product', -0.32055623673442618),
 ('less', -0.20956286453453807),
 ('car', 0.05885467115273034),
 ('old', 0.085512779463054259),
 ('able', 0.19090857206454992),
 ('well', 0.50376045776762213),
 ('little', 0.52018576271821038),
 ('great', 0.9439995905719335),
 ('easy', 1.1925382734895196),
 ('love', 1.3636897593113817),
 ('perfect', 1.5098124766922481),
 ('loves', 1.673073892593705)]

**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

In [33]:
sentiment_model_words = vectorizer.get_feature_names()
print("Number of words in the original model ", len(sentiment_model_words))

Number of words in the original model  121712


In [34]:
matching_indices = [sentiment_model_words.index(x) for x in significant_words]

sentimental_model_coef = sentiment_model.coef_[0]
coef_matching_indices = [sentimental_model_coef[x] for x in matching_indices]

In [35]:
##Checking whether the coefficents have same signature in both the models
simple_model_coefficients = simple_model.coef_[0]
coef_matching_indices = [(simple_model_coefficients[x] < 0) == (coef_matching_indices[x] < 0) for x in range(0,20)]

# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [36]:
train_data_predictions = sentiment_model.predict(train_matrix)
train_data_count = len(train_data_predictions)
print("Train data count", train_data_count)
accurate_predictions = np.sum(train_data['sentiment'] == train_data_predictions)
print("Accuracy for sentiment_model on train_data", accurate_predictions/train_data_count)

Train data count 133416
Accuracy for sentiment_model on train_data 0.968504527193


Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [37]:
train_data_predictions = simple_model.predict(train_matrix_word_subset)
accurate_predictions = np.sum(train_data['sentiment'] == train_data_predictions)
print("Accuracy for simple_model on train_data", accurate_predictions/train_data_count)

Accuracy for simple_model on train_data 0.866822570007


**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?

Now, we will repeat this exercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [38]:
test_data_predictions = sentiment_model.predict(test_matrix)
test_data_count = len(test_data_predictions)
print("Test data count", test_data_count)
accurate_predictions = np.sum(test_data['sentiment'] == test_data_predictions)
print("Accuracy for sentiment_model on test_data", accurate_predictions/test_data_count)

Test data count 33336
Accuracy for sentiment_model on test_data 0.932295416367


Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [39]:
test_data_predictions = simple_model.predict(test_matrix_word_subset)

accurate_predictions = np.sum(test_data['sentiment'] == test_data_predictions)
print("Accuracy for simple_model on test_data", accurate_predictions/test_data_count)

Accuracy for simple_model on test_data 0.869360451164


**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set?

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [40]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print(num_positive)
print(num_negative)

112164
21252


In [41]:
print("Train data accuracy for majority class classifier", num_positive/len(train_data))

Train data accuracy for majority class classifier 0.840708760568


Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [42]:
num_positive  = (test_data['sentiment'] == +1).sum()
num_negative = (test_data['sentiment'] == -1).sum()
print(num_positive)
print(num_negative)

28095
5241


**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)?

In [43]:
print("Test data accuracy for majority class classifier", num_positive/len(test_data))

Test data accuracy for majority class classifier 0.842782577394
